## Startup

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from hmmlearn import hmm

import os
import pickle

In [5]:
random_state=42
np.random.seed(random_state)

## Data Retrieval

In [13]:
dataroute=os.path.join("..",  "data")
dumproute=os.path.join("..",  "dump")
resultsroute=os.path.join("..",  "results")

In [14]:
with open(os.path.join(dumproute, "tickerlist.pickle"), 'rb') as f:
    tickerlist=pickle.load(f)

In [23]:
start='2013-01-01'
end="2023-06-01"

name=f'finaldf_dataset_{start}_{end}.pickle'
filename=os.path.join(dataroute, name)
with open(filename, 'rb') as handle:
    df=pickle.load(handle)


In [26]:
df.head(2)

,^MERV_rets,^MERV_log_rets,^MERV_gk_vol,GGAL_rets,GGAL_log_rets,GGAL_gk_vol,GGAL.BA_rets,GGAL.BA_log_rets,GGAL.BA_gk_vol,YPF_rets,...,BMA_gk_vol,BMA.BA_rets,BMA.BA_log_rets,BMA.BA_gk_vol,USD_rets,USD_log_rets,USD_gk_vol,USD_^MERV_rets,USD_^MERV_log_rets,USD_^MERV_gk_vol
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-03,0.007552,0.007524,0.000129,-0.012748,-0.012830,0.001228,0.010616,0.010560,0.000677,-0.004016,...,0.000557,-0.007812,-0.007843,0.000857,0.011942,0.011871,0.000020,-0.002186,-0.002188,0.000129
2013-01-04,0.007092,0.007067,0.000158,-0.010043,-0.010094,0.000554,-0.006303,-0.006323,0.000208,-0.000672,...,0.000695,-0.027559,-0.027946,0.000692,0.021496,0.021269,0.000228,-0.008133,-0.008167,0.000158


## HMM Training

In [37]:
stock="GGAL.BA"
datacols=[f'{stock}_log_rets', f"{stock}_gk_vol"]
X=df[datacols].values.reshape(-1, len(datacols))
model=hmm.GaussianHMM(n_components=2, covariance_type="diag", n_iter=50, random_state=random_state)
result=model.fit(X)
predict=model.predict(X)

In [46]:
type(result)

hmmlearn.hmm.GaussianHMM

In [53]:
range=range(1,11)
emptydf=pd.DataFrame(columns=["AIC", "BIC"], index=range)
emptydf.fillna(np.inf)
results_dict_df={stock:emptydf for stock in tickerlist}

In [54]:
aic_best_model={stock:None for stock in tickerlist}
bic_best_model={stock:None for stock in tickerlist}

aic_best_residuals={stock:None for stock in tickerlist}
bic_best_residuals={stock:None for stock in tickerlist}

In [55]:
for stock in tickerlist:
    columns = [f'{stock}_log_rets', f'{stock}_gk_vol']
    insample_data = df[columns]

    param_dict={
        "covariance_type" : "diag", 
        "n_iter" : 500,
        "random_state" : random_state
        #no voy a usar startprob_prior por devlog 20-06-23
        }

    for nstate in range:
        print(stock, nstate)
        model = hmm.GaussianHMM(n_components= nstate, **param_dict)
        results = model.fit(insample_data)

        results_dict_df[stock].loc[nstate, "AIC"]=model.aic(insample_data)
        results_dict_df[stock].loc[nstate, "BIC"]=model.bic(insample_data)

Model is not converging.  Current: 18846.005278247954 is not greater than 18847.134444221098. Delta is -1.1291659731432446
Model is not converging.  Current: 18142.20139769384 is not greater than 18142.85021337392. Delta is -0.6488156800805882


^MERV 1
^MERV 2
^MERV 3
^MERV 4


Model is not converging.  Current: 18905.499058655805 is not greater than 18910.228067216696. Delta is -4.729008560891089


^MERV 5


Model is not converging.  Current: 18180.69639406222 is not greater than 18181.27568431714. Delta is -0.5792902549219434
Model is not converging.  Current: 18905.19071950915 is not greater than 18910.003613474557. Delta is -4.812893965405237


...

GGAL 1
GGAL 2
GGAL 3
GGAL 4


Model is not converging.  Current: 17414.153727044355 is not greater than 17414.567559005332. Delta is -0.4138319609774044
Model is not converging.  Current: 16868.16364769593 is not greater than 16870.550928025812. Delta is -2.3872803298836516


GGAL 5
GGAL 6


c:\Users\Alfred\.julia\conda\3\envs\hmm\Lib\site-packages\hmmlearn\hmm.py:340: RuntimeWarning: invalid value encountered in divide
  self.means_ = ((means_weight * means_prior + stats['obs'])
Some rows of transmat_ have zero sum because no transition from the state was ever observed.
Some rows of transmat_ have zero sum because no transition from the state was ever observed.
Some rows of transmat_ have zero sum because no transition from the state was ever observed.
Some rows of transmat_ have zero sum because no transition from the state was ever observed.
Some rows of transmat_ have zero sum because no transition from the state was ever observed.
Some rows of transmat_ have zero sum because no transition from the state was ever observed.
Some rows of transmat_ have zero sum because no transition from the state was ever observed.
Some rows of transmat_ have zero sum because no transition from the state was ever observed.
Some rows of transmat_ have zero sum because no transition from 

ValueError: startprob_ must sum to 1 (got nan)

In [ ]:
for stock in tickerlist:
    best_aic_nstate=results_dict_df[stock]["AIC"].astype(float).idxmin()
    best_bic_nstate=results_dict_df[stock]["BIC"].astype(float).idxmin()

    aic_best_model[stock]=hmm.GaussianHMM(n_components = best_aic_nstate, **param_dict).fit(insample_data)
    bic_best_model[stock]=hmm.GaussianHMM(n_components = best_bic_nstate, **param_dict).fit(insample_data)

    # residuals is observed minus predicted
    aic_best_residuals[stock]= X - aic_best_model[stock].predict(X)
    bic_best_residuals[stock]= X - aic_best_model[stock].predict(X)

In [32]:
results_dict_df

{'^MERV':                                                   AIC  \
 1   <bound method BaseHMM.aic of GaussianHMM(n_ite...   
 2   <bound method BaseHMM.aic of GaussianHMM(n_com...   
 3   <bound method BaseHMM.aic of GaussianHMM(n_com...   
 4   <bound method BaseHMM.aic of GaussianHMM(n_com...   
 5   <bound method BaseHMM.aic of GaussianHMM(n_com...   
 6   <bound method BaseHMM.aic of GaussianHMM(n_com...   
 7   <bound method BaseHMM.aic of GaussianHMM(n_com...   
 8   <bound method BaseHMM.aic of GaussianHMM(n_com...   
 9   <bound method BaseHMM.aic of GaussianHMM(n_com...   
 10  <bound method BaseHMM.aic of GaussianHMM(n_com...   
 
                                                   BIC  
 1   <bound method BaseHMM.bic of GaussianHMM(n_ite...  
 2   <bound method BaseHMM.bic of GaussianHMM(n_com...  
 3   <bound method BaseHMM.bic of GaussianHMM(n_com...  
 4   <bound method BaseHMM.bic of GaussianHMM(n_com...  
 5   <bound method BaseHMM.bic of GaussianHMM(n_com...  
 6   <bou

# Guardado de datos

In [ ]:
with open(os.path.join(resultsroute, "HMM_univ_aic_bestmodels.pickle"), "wb") as output_file:
    pickle.dump(aic_best_model, output_file)

with open(os.path.join(resultsroute, "HMM_univ_bic_bestmodels.pickle"), "wb") as output_file:
    pickle.dump(bic_best_model, output_file)

Los modelos sirven los residuos NO!
https://github.com/alfsn/regime-switching-hmm/issues/27

In [ ]:
with open(os.path.join(resultsroute, "HMM_univ_aic_residuals.pickle"), "wb") as output_file:
    pickle.dump(aic_best_residuals, output_file)

with open(os.path.join(resultsroute, "HMM_univ_bic_residuals.pickle"), "wb") as output_file:
    pickle.dump(bic_best_residuals, output_file)

In [5]:
models={}
comps=[2,3,4]
datacols=["log_rets","gk_vol"]

for key in data.keys():
    print(key)
    for comp in comps:
        print(comp)
        # TODO: Change from single modelname to [key][comp] nested dict
        modelname=f"{key}_{comp}_model"
        predictionname=f"{key}_{comp}_prediction"
        
        X = data[key][datacols].values.reshape(-1, len(datacols))
        # bivariate
        # log returns and intraday volatility        
        models[modelname]=hmm.GaussianHMM(n_components = comp, #no voy a usar startprob_prior por devlog 20-06-23
                                          covariance_type = "diag", 
                                          n_iter = 500,
                                          random_state = random_state)
        models[modelname].fit(X)
        models[predictionname]=models[modelname].predict(X)

^MERV
2


Model is not converging.  Current: 18846.005278247954 is not greater than 18847.134444221098. Delta is -1.1291659731432446
Model is not converging.  Current: 18142.20139769384 is not greater than 18142.85021337392. Delta is -0.6488156800805882
Model is not converging.  Current: 18905.499058655805 is not greater than 18910.228067216696. Delta is -4.729008560891089


3
4
GGAL
2


Model is not converging.  Current: 18038.2624343628 is not greater than 18038.278369587973. Delta is -0.015935225172142964


3


...

4
USD
2
3


Model is not converging.  Current: 21535.656358535536 is not greater than 21535.89830316691. Delta is -0.24194463137246203
Model is not converging.  Current: 19946.186981522354 is not greater than 19948.99284249492. Delta is -2.805860972566734


4
USD_^MERV
2
3


Model is not converging.  Current: 20007.553148425937 is not greater than 20007.581922967158. Delta is -0.028774541220627725


4


In [6]:
# Predict the hidden states corresponding to observed X.
for key in data.keys():
    for comp in comps:
        print(">"*30, key)
        model=models[f"{key}_{comp}_model"]
        prediction=models[f"{key}_{comp}_prediction"]
        print("unique states: ", pd.unique(prediction))
        print("\nStart probabilities:")
        print(model.startprob_)
        print("\nTransition matrix:")
        print(model.transmat_)
        print("\nGaussian distribution means:")
        print(model.means_)
        print("\nGaussian distribution covariances:")
        print(model.covars_)
        print()

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ^MERV
unique states:  [0 1]

Start probabilities:
[9.99999992e-01 7.77089851e-09]

Transition matrix:
[[0.99190044 0.00809956]
 [0.64107689 0.35892311]]

Gaussian distribution means:
[[ 0.0023665   0.00062083]
 [-0.03635524  0.0146091 ]]

Gaussian distribution covariances:
[[[4.65243073e-04 0.00000000e+00]
  [0.00000000e+00 4.63173289e-06]]

 [[1.44224149e-02 0.00000000e+00]
  [0.00000000e+00 1.50614206e-03]]]

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ^MERV
unique states:  [1 0 2]

Start probabilities:
[2.64082753e-02 9.73591725e-01 1.62699196e-11]

Transition matrix:
[[1.36233321e-03 9.98614722e-01 2.29446306e-05]
 [9.77992322e-01 1.06337631e-02 1.13739147e-02]
 [2.70638123e-01 2.45724508e-01 4.83637369e-01]]

Gaussian distribution means:
[[ 0.00147874  0.0006487 ]
 [ 0.00320576  0.00061345]
 [-0.03997792  0.01558215]]

Gaussian distribution covariances:
[[[5.03860880e-04 0.00000000e+00]
  [0.00000000e+00 8.81549832e-06]]

 [[4.56882297e-04 0.00000000e+00]
  [0.00

In [7]:
def plot_close_rets_vol(data, key, comp):
    model=models[f"{key}_{comp}_model"]
    prediction=models[f"{key}_{comp}_prediction"]
    states=set(prediction)

    fig=plt.figure(figsize = (20, 20))
    plt.tight_layout()
    plt.title(f"{key} Close, Log returns and intraday Vol\n{comp} states")

    for subplot, var in zip(range(1,4), ["Close", "log_rets", "gk_vol"]):    
        plt.subplot(3,1,subplot)
        for i in set(prediction):
            state = (prediction == i)
            x = data[key].index[state]
            y = data[key][var].iloc[state]
            plt.plot(x, y, '.')
        plt.legend(states, fontsize=16)
        
        plt.grid(True)
        plt.xlabel("datetime", fontsize=16)
        plt.ylabel(var, fontsize=16)
            
    plt.savefig(os.path.join(resultsroute, "graphs", 
                             f"{comp}_states", 
                             f"{key}_model_{comp}.png"))

In [ ]:
for key in data.keys():
    for comp in comps:
        plot_close_rets_vol(data, key, comp)

## Coefficients 

In [9]:
models.keys()

dict_keys(['^MERV_2_model', '^MERV_2_prediction', '^MERV_3_model', '^MERV_3_prediction', '^MERV_4_model', '^MERV_4_prediction', 'GGAL_2_model', 'GGAL_2_prediction', 'GGAL_3_model', 'GGAL_3_prediction', 'GGAL_4_model', 'GGAL_4_prediction', 'GGAL.BA_2_model', 'GGAL.BA_2_prediction', 'GGAL.BA_3_model', 'GGAL.BA_3_prediction', 'GGAL.BA_4_model', 'GGAL.BA_4_prediction', 'YPF_2_model', 'YPF_2_prediction', 'YPF_3_model', 'YPF_3_prediction', 'YPF_4_model', 'YPF_4_prediction', 'YPFD.BA_2_model', 'YPFD.BA_2_prediction', 'YPFD.BA_3_model', 'YPFD.BA_3_prediction', 'YPFD.BA_4_model', 'YPFD.BA_4_prediction', 'EDN_2_model', 'EDN_2_prediction', 'EDN_3_model', 'EDN_3_prediction', 'EDN_4_model', 'EDN_4_prediction', 'EDN.BA_2_model', 'EDN.BA_2_prediction', 'EDN.BA_3_model', 'EDN.BA_3_prediction', 'EDN.BA_4_model', 'EDN.BA_4_prediction', 'BMA_2_model', 'BMA_2_prediction', 'BMA_3_model', 'BMA_3_prediction', 'BMA_4_model', 'BMA_4_prediction', 'BMA.BA_2_model', 'BMA.BA_2_prediction', 'BMA.BA_3_model', 'BMA.B

In [10]:
for model in models.keys()
    print(model)
    print("Matriz de transicion:")
    print(model.transmat_)
    print("Matriz de emisiones:")
    print(model.means_)

SyntaxError: expected ':' (2188482437.py, line 1)

## HMM Selection

Selecting the Number of States in Hidden Markov Models: Pragmatic Solutions Illustrated Using Animal Movement
https://sci-hub.st/10.1007/s13253-017-0283-8